In [1]:
import matplotlib.pyplot as plt #library for visualizing data
%matplotlib widget 
#setting for jupyter lab
plt.rcParams['figure.figsize'] = [10, 5] #setting figure size (plots)

import pandas as pd  # (software library for data analysis and manipulation, https://pandas.pydata.org/docs/)
import numpy as np  # (software library for matrix multiplications, https://numpy.org/doc/)
from numpy.random import randint
from numpy.random import rand
import statistics as stats  # (python module for statistic calculations, https://docs.python.org/3/library/statistics.html)
import time #python time module
import math #python module for mathematical operations


In [2]:
def fitness(x):
    '''fitness function to determine quality of solutions'''
    des_result = 0
    fitness =  abs(des_result - (x[0]-3)**2.0 + math.log(x[1]) + math.sin(x[2]) - (x[3]+1)*x[0])
    return fitness
 
def decode(number_of_variables, n_bits, bitstring, ranges):
    '''decode bitstring to numbers'''
    decoded = list()
    largest = 2**n_bits
    for i in range(len(ranges)):
        
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        
        chars = ''.join([str(s) for s in substring]) # convert bitstring to a string of chars
        integer = int(chars, 2) #convert string to int
        
        value = ranges[i][0] + (integer/largest) * (ranges[i][1] - ranges[i][0])
        
        decoded.append(value) #store
        
    return decoded
 
def selection(pop, scores, k=3):
    '''select individuals for reproduction selection'''
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]
 
    
def crossover(p1, p2, r_cross):
    '''crossover two parents to create two children'''
    c1, c2 = p1.copy(), p2.copy() # children are copies of parents by default

    # check for recombination
    if rand() < r_cross:
        pt = randint(1, len(p1)-2) # select crossover point that is not on the end of the string
        
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]
 
    
def mutation(bitstring, r_mut):
    '''apply mutation to bitsring'''
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            bitstring[i] = 1 - bitstring[i] # flip the bit



In [19]:
ranges = [[-10000, 10000],[0.00001, 10000],[-10000, 10000],[-10000, 10000]]
n_iterations = 500
n_bits = 8 #bits per variable
n_population = 500
r_cross = 0.2 # crossover rate
r_mut = 70 / (float(n_bits) * len(ranges)) # mutation rate

In [20]:
population = [randint(0, 2, n_bits*len(ranges)).tolist() for _ in range(n_population)] #generate population
best, best_eval = 0, fitness(decode(len(ranges), n_bits, population[0], ranges))

display(pd.DataFrame(population))

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1,0,0,0,1,0,1,1,1,1,...,0,0,1,1,0,0,1,0,1,0
1,1,1,1,1,1,1,0,0,0,0,...,0,1,1,0,0,0,1,0,1,0
2,1,1,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,1,1
3,1,0,1,1,0,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,1
4,0,1,0,1,1,0,1,1,0,1,...,1,0,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,1,1,0,0,1,1,0,0,...,1,0,0,1,0,1,0,1,1,1
496,0,0,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
497,1,0,1,1,0,1,1,0,1,1,...,1,0,1,0,1,1,0,1,1,0
498,0,0,0,0,1,0,0,0,0,1,...,1,1,1,1,0,0,0,0,1,1


In [21]:
history = []

for gen in range(n_iterations):
        decoded = [decode(len(ranges), n_bits, p, ranges) for p in population] # decode population
        scores = [fitness(d) for d in decoded] # evaluate all candidates in the population
        
        # check for new best solution
        for i in range(n_population):
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print(f'''>{gen}< New best fit: f({decoded[i]}) = {scores[i]}''')
                
        selected = [selection(population, scores) for _ in range(n_population)] # select parents
        #print(len(selected))
        #selected = pd.DataFrame([population, scores]).T.sort_values(by=1).head(200)[0].to_numpy()

        # create the next generation
        children = list()
        for i in range(0, n_population, 2):

            p1, p2 = selected[i], selected[i+1] # get selected parents in pairs
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                mutation(c, r_mut) # mutation
                children.append(c) # store for next generation

        population = children # replace population

        history.append([gen, best_eval, stats.mean(scores), stats.median(scores)]) #store history
        
print('Finished.\n')
decoded = decode(len(ranges), n_bits, best, ranges)
print(f'''Best fit: f({decoded}) = {best_eval}''')

>0< New best fit: f([3906.25, 117.1875098828125, -8671.875, -4609.375]) = 2766108.163408922
>0< New best fit: f([-234.375, 117.1875098828125, 4609.375, 4296.875]) = 950971.7142830825
>0< New best fit: f([156.25, 2773.437507226563, 6328.125, 5000.0]) = 804883.0669620362
>0< New best fit: f([-156.25, 8164.062501835938, 2578.125, 2734.375]) = 402051.6899637441
>0< New best fit: f([312.5, 1015.625008984375, 8671.875, 234.375]) = 169337.1326241481
>0< New best fit: f([-78.125, 8359.375001640625, -1093.75, 1093.75]) = 78954.6512528314
>0< New best fit: f([0.0, 2812.5000071875, -3906.25, -6093.75]) = 0.10908182667290534
>1< New best fit: f([0.0, 6015.625003984375, 2890.625, -4140.625]) = 0.05416391924790459
>5< New best fit: f([0.0, 4921.875005078125, -2265.625, -234.375]) = 0.012744811204707718
Finished.

Best fit: f([0.0, 4921.875005078125, -2265.625, -234.375]) = 0.012744811204707718


In [23]:
history_pd = pd.DataFrame(history, columns = ['gen', 'fittest', 'mean_fit', 'median_fit']).set_index('gen')

plt.close('all')
history_pd['fittest'].plot(title='Fitness over generations', ylim=[0, history_pd['fittest'].max()*1.1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'Fitness over generations'}, xlabel='gen'>

In [24]:
plt.close('all')
history_pd[['mean_fit', 'median_fit']].plot(title = 'Mean and median fitness over generations', ylim=[0, history_pd['mean_fit'].max()*1.1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'Mean and median fitness over generations'}, xlabel='gen'>